In [2]:
from pathlib import Path
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import torch

sys.path.insert(0, "./DiffRoute")
sys.path.insert(0, "./DiffHydro")

from diffhydro import LTIStagedRouter, StagedCatchmentInterpolator, TimeSeriesThDF
from diffhydro.utils import Timer, nse_fn
from diffhydro.io import read_rapid_graph, read_multiple_rapid_graphs

sync = torch.cuda.synchronize
%load_ext line_profiler

### Experiment parameters

In [3]:
# graph partitioning parameters
plength_thr = 10**4
node_thr = 10**4
# Routing model parameterss
max_delay = 32
dt = 1/24
# Experiment pathes and variables
device = "cuda:0"
root = Path("/data_prediction005/SYSTEM/prediction002/home/tristan/data/geoflow/") # Set your data root path

### Download data if necessary

In [ ]:
import sys; sys.path.insert(0, "..") # Needed to access the download utilities
from utils.download import download_rapid_config, 

In [4]:
discharge_path = root / "retro_feather"
vpu_config_path = root / 'data' / 'configs'
runoff_path = root / "data" / "daily_sparse_runoff.feather"
interp_weight_path = root / "data" / "interp_weight.pkl"

In [7]:
%time all_runoff = pd.read_feather(runoff_path).loc[:"2019"]  / (3600. * 24) # Convert in m3 / s
%time all_interp_df = pd.read_pickle(interp_weight_path).set_index("river_id")

CPU times: user 58.3 s, sys: 2min 47s, total: 3min 46s
Wall time: 36.5 s
CPU times: user 159 ms, sys: 648 ms, total: 807 ms
Wall time: 806 ms


In [14]:
g = read_rapid_graph(vpu_config_path / "305")
nodes = list(g.g.nodes)
interp_305 = all_interp_df.loc[nodes]
runoff_305 = all_runoff[interp_305["pixel_idx"].unique()]

In [27]:
runoff_305.to_pickle("runoff_pixel_305.pkl")
interp_305.to_pickle("interp_weight_305.pkl")

### Evaluation per VPU against GEOGLOWS

In [11]:
t = Timer(device)

for i,vpu in enumerate(tqdm(list(discharge_path.glob("*.feather")))):
    
    with t("Load Q"): q = pd.read_feather(vpu)
    with t("GPU Q") : lbl = TimeSeriesThDF.from_pandas(q).to(device)
    with t("Load G"): g = read_rapid_graph(vpu_config_path / vpu.stem, 
                                           plength_thr=plength_thr, 
                                           node_thr=node_thr).to(device)
    with t("Filter"):
        interp_df = all_interp_df.loc[g.nodes]
        pix_idxs = interp_df["pixel_idx"].unique()
        runoff = TimeSeriesThDF.from_pandas(all_runoff[pix_idxs]).to(device)
    
    with t("Cat init"):   cat = StagedCatchmentInterpolator(g, runoff, interp_df).to(device)
    with t("Model init"): model = LTIStagedRouter(
                                        max_delay=max_delay,
                                        block_size=block_size,
                                        block_f=block_f,
                                        dt=dt,
                                        cascade=cascade,
                                        sampling_mode=sampling_mode
                                    ).to(device)

    with t("Cat interp"): runoffs = cat.yield_all_runoffs(runoff)
        
    with t("Model eval"):
        for output in model.route_all_clusters_yield(runoffs, g, 
                                                     display_progress=False):
            with t("Index lbl"): y = lbl[output.columns]
            with t("Loss"): nse = nse_fn(output.values, y.values)
            with t("nse cpu"): 
                nses.append(pd.Series(nse.squeeze().cpu().numpy(), index=output.columns))

  0%|          | 0/125 [00:00<?, ?it/s]

#### Upstream stats computations ... ####


Computing breakpoints:   0%|          | 0/34443 [00:00<?, ?it/s]

#### Segmentation into subgraphs ... ####
Removing edges...


  0%|          | 0/34443 [00:00<?, ?it/s]

...

Computing merged graphs node idxs...


  0%|          | 0/94 [00:00<?, ?it/s]

Match breakpoint nodes across clusters...


  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

In [6]:
pd.concat(nses).median().item()

0.9996680021286011

### Full model evaluation fast

In [ ]:
t = Timer(device)
g = read_multiple_rapid_graphs(vpu_config_path.glob("*"), 
                               plength_thr=plength_thr, 
                               node_thr=node_thr)
interp_df = all_interp_df.reset_index()
runoff = all_runoff

In [ ]:
with t("Cat init"):   cat = CatchmentInterpolator(g, runoff, interp_df).to(device)
with t("Model init"): model = LTIStagedRouter(max_delay=max_delay, dt=dt).to(device)

with t("Cat Interp"):
    for runoff in cat.yield_all_runoffs():
        pass

In [ ]:
inp_runoffs = cat.yield_all_runoffs()
out_discharge = model.route_all_clusters_yield(inp_runoffs, g)

In [ ]:
with t("Model eval"):
    for output in tqdm(out_discharge):
        pass

In [ ]:
df, tot = t.summary()
df